In [1]:
import sys
import os
from typing import Dict, List, Optional, Tuple
import pandas as pd
import numpy as np
from datetime import datetime
import logging

# Add the project root directory to Python path
project_root = r'c:\Users\pedro.hun\Documents\repos\option_monitor2'
if project_root not in sys.path:
    sys.path.insert(0, project_root)


from src.core.entities.option import Option, OptionType
from src.core.entities.iv_calc import IVCalc
from src.core.interpolation_single_expiry.svi import SVI
from src.data_fetchers.excel_reader import ExcelReader, ExcelReaderConfig
from src.data_fetchers.holiday_df_reader import Holidays
from src.filters.option_filters import option_data_positive, intrinsic_lower_than_price, option_lower_than_underlying, valid_price, valid_obj_func, valid_iv
from src.utils.option_data_calc import get_type, calculate_tte_days, calculate_tte_years, calculate_mid_price, calculate_bid_ask_spread, calculate_relative_spread, calculate_forward_price, calculate_moneyness, calculate_log_moneyness, calculate_intrinsic_value
from src.orchestrators.df_creator import DataFrameCreator, BasicFilters, DFCreator
from src.core.entities.black_scholes import w, d1, d2, discount_factor, black_scholes


In [2]:
df = DFCreator()
final_df = df.apply_iv_calc()

In [3]:
final_df

,bid,ask,lastPrice,Strike,openInterest,volume,ticker,Expiry,IV,SpotPrice,...,TTE_years,MidPrice,Spread,RelativeSpread,ForwardPrice,Moneyness,LogMoneyness,IntrinsicValue,BSPrice,CalcIV
14,0.48,0.63,0.57,29.40,141300.0,140000.0,PETRJ300W4,2025-10-24,0.192451,29.73,...,0.011905,0.555,0.15,27.027027,29.779507,1.012908,0.012826,0.379507,0.307901,0.343327
15,0.40,0.45,0.43,29.65,169100.0,214600.0,PETRJ302W4,2025-10-24,0.203905,29.73,...,0.011905,0.425,0.05,11.764706,29.779507,1.004368,0.004358,0.129507,0.312895,0.287376
16,0.26,0.29,0.30,29.90,680800.0,894000.0,PETRJ305W4,2025-10-24,0.202827,29.73,...,0.011905,0.275,0.03,10.909091,29.779507,0.995970,-0.004038,0.000000,0.189807,0.266158
17,0.05,0.22,0.19,30.15,584500.0,1036200.0,PETRJ307W4,2025-10-24,0.196075,29.73,...,0.011905,0.135,0.17,125.925926,29.779507,0.987712,-0.012364,0.000000,-0.049653,0.306853
18,0.11,0.13,0.12,30.40,367600.0,1289700.0,PETRJ310W4,2025-10-24,0.197220,29.73,...,0.011905,0.120,0.02,16.666667,29.779507,0.979589,-0.020622,0.000000,-0.351876,0.444971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757,0.53,0.87,0.78,34.25,13400.0,3600.0,PETRC342,2026-03-20,0.218525,29.73,...,0.408730,0.700,0.34,48.571429,31.477769,0.919059,-0.084405,0.000000,-0.461927,0.330647
1758,0.21,1.50,0.23,38.25,91700.0,1000.0,PETRC480,2026-03-20,0.229859,29.73,...,0.408730,0.855,1.29,150.877193,31.477769,0.822948,-0.194862,0.000000,-4.999352,0.680263
1814,5.00,10.90,5.78,26.25,115500.0,200.0,PETRD360,2026-04-17,0.282393,29.73,...,0.484127,7.950,5.90,74.213836,31.811223,1.211856,0.192153,5.561223,2.176970,0.849026
1850,0.40,2.50,0.90,35.25,145000.0,300.0,PETRD449,2026-04-17,0.237370,29.73,...,0.484127,1.450,2.10,144.827586,31.811223,0.902446,-0.102646,0.000000,-0.652084,0.426885
